In [21]:
# Import necessary libraries
import cv2
import numpy as np
import os
import time
from keras.models import load_model
from pygame import mixer


In [22]:
# Initialize sound mixer and load alarm sound
mixer.init()
sound = mixer.Sound('alarm.mp3')  # Make sure 'alarm.wav' is in the same folder


In [23]:
# Load Haar Cascade classifiers for face and eyes
rpred = 1  # Assume open
lpred = 1  # Assume open

face = cv2.CascadeClassifier('haar cascade files/haarcascade_frontalface_alt.xml')
leye = cv2.CascadeClassifier('haar cascade files/haarcascade_lefteye_2splits.xml')
reye = cv2.CascadeClassifier('haar cascade files/haarcascade_righteye_2splits.xml')

# Load the trained CNN model for eye state detection
model = load_model('models/cnncat2.h5')


In [48]:
# Initialize webcam and constants
labels = ['Closed', 'Open']
cap = cv2.VideoCapture(0)  # Start webcam capture
font = cv2.FONT_HERSHEY_COMPLEX_SMALL

# Initialize control variables
frame_counter = 0         # Counts frames processed
score = 0                 # Eye closure score
thickness = 2             # Thickness for alert rectangle
rpred = [99]              # Right eye prediction placeholder
lpred = [99]              # Left eye prediction placeholder
alarm_on = False          # Alarm state flag
project_path = os.getcwd()  # Current working directory


In [49]:
while True:
    ret, frame = cap.read()
    if not ret:
        print("❌ Failed to capture frame from webcam.")
        break

    height, width = frame.shape[:2]
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
    left_eye = leye.detectMultiScale(gray)
    right_eye = reye.detectMultiScale(gray)

    rpred = 1  # Default to open
    lpred = 1

    for (x, y, w, h) in right_eye:
        r_eye = frame[y:y+h, x:x+w]
        r_eye = cv2.cvtColor(r_eye, cv2.COLOR_BGR2GRAY)
        r_eye = cv2.resize(r_eye, (24, 24))
        r_eye = r_eye / 255.0
        r_eye = r_eye.reshape(1, 24, 24, 1)
        rpred = model.predict(r_eye, verbose=0).argmax(axis=1)[0]
        break

    for (x, y, w, h) in left_eye:
        l_eye = frame[y:y+h, x:x+w]
        l_eye = cv2.cvtColor(l_eye, cv2.COLOR_BGR2GRAY)
        l_eye = cv2.resize(l_eye, (24, 24))
        l_eye = l_eye / 255.0
        l_eye = l_eye.reshape(1, 24, 24, 1)
        lpred = model.predict(l_eye, verbose=0).argmax(axis=1)[0]
        break

    if rpred == 0 and lpred == 0:
        score += 1
        cv2.putText(frame, "Eyes Closed", (10, height - 20), font, 1, (0, 0, 255), 2)
    else:
        score = 0
        cv2.putText(frame, "Eyes Open", (10, height - 20), font, 1, (0, 255, 0), 2)
        if alarm_on:
            sound.stop()
            alarm_on = False

    if score >= 5:
        if not alarm_on:
            sound.play(-1)
            alarm_on = True

    cv2.putText(frame, f"Score: {score}", (400, height - 20), font, 1, (255, 255, 255), 1)
    cv2.imshow('Driver Drowsiness Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
sound.stop()
